In [1]:
import torch
from torch import nn
from collections import OrderedDict

def conv_batch(in_num, out_num, kernel_size=3, padding=1, stride=1):
    return nn.Sequential(
        nn.Conv2d(in_num, out_num, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
        nn.BatchNorm2d(out_num),
        nn.LeakyReLU())


# Residual block
class DarkResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(DarkResidualBlock, self).__init__()

        reduced_channels = int(in_channels/2)
        self.layer1 = conv_batch(in_channels, reduced_channels, kernel_size=1, padding=0)
        self.layer2 = conv_batch(reduced_channels, in_channels)

    def forward(self, x):
        residual = x

        out = self.layer1(x)
        out = self.layer2(out)
        out += residual
        return out


class Darknet53(nn.Module):
    def __init__(self, block, num_classes):
        super(Darknet53, self).__init__()

        self.num_classes = num_classes

        self.conv1 = conv_batch(3, 32)
        self.conv2 = conv_batch(32, 64, stride=2)
        self.residual_block1 = self.make_layer(block, in_channels=64, num_blocks=1)
        self.conv3 = conv_batch(64, 128, stride=2)
        self.residual_block2 = self.make_layer(block, in_channels=128, num_blocks=2)
        self.conv4 = conv_batch(128, 256, stride=2)
        self.residual_block3 = self.make_layer(block, in_channels=256, num_blocks=8)
        self.conv5 = conv_batch(256, 512, stride=2)
        self.residual_block4 = self.make_layer(block, in_channels=512, num_blocks=8)
        self.conv6 = conv_batch(512, 1024, stride=2)
        self.residual_block5 = self.make_layer(block, in_channels=1024, num_blocks=4)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, self.num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.residual_block1(out)
        out = self.conv3(out)
        out = self.residual_block2(out)
        out = self.conv4(out)
        out = self.residual_block3(out)
        out = self.conv5(out)
        out = self.residual_block4(out)
        out = self.conv6(out)
        out = self.residual_block5(out)
        out = self.global_avg_pool(out)
        out = out.view(-1, 1024)
        out = self.fc(out)

        return out

    def represent_model(self) :
        self.features = nn.Sequential(
            self.conv1,
            self.conv2,
            self.residual_block1[0].layer1,
            self.residual_block1[0].layer2,
            self.conv3,
            self.residual_block2[0].layer1,
            self.residual_block2[0].layer2,
            self.residual_block2[1].layer1,
            self.residual_block2[1].layer2,
            self.conv4,
            self.residual_block3[0].layer1,
            self.residual_block3[0].layer2,
            self.residual_block3[1].layer1,
            self.residual_block3[1].layer2,
            self.residual_block3[2].layer1,
            self.residual_block3[2].layer2,
            self.residual_block3[3].layer1,
            self.residual_block3[3].layer2,
            self.residual_block3[4].layer1,
            self.residual_block3[4].layer2,
            self.residual_block3[5].layer1,
            self.residual_block3[5].layer2,
            self.residual_block3[6].layer1,
            self.residual_block3[6].layer2,
            self.residual_block3[7].layer1,
            self.residual_block3[7].layer2,
            self.conv5,
            self.residual_block4[0].layer1,
            self.residual_block4[0].layer2,
            self.residual_block4[1].layer1,
            self.residual_block4[1].layer2,
            self.residual_block4[2].layer1,
            self.residual_block4[2].layer2,
            self.residual_block4[3].layer1,
            self.residual_block4[3].layer2,
            self.residual_block4[4].layer1,
            self.residual_block4[4].layer2,
            self.residual_block4[5].layer1,
            self.residual_block4[5].layer2,
            self.residual_block4[6].layer1,
            self.residual_block4[6].layer2,
            self.residual_block4[7].layer1,
            self.residual_block4[7].layer2,
            self.conv6,
            self.residual_block5[0].layer1,
            self.residual_block5[0].layer2,
            self.residual_block5[1].layer1,
            self.residual_block5[1].layer2,
            self.residual_block5[2].layer1,
            self.residual_block5[2].layer2,
            self.residual_block5[3].layer1,
            self.residual_block5[3].layer2)

        self.classifier = nn.Sequential(nn.Sequential(self.global_avg_pool, self.fc))

    def make_layer(self, block, in_channels, num_blocks):
        layers = []
        for i in range(0, num_blocks):
            layers.append(block(in_channels))
        return nn.Sequential(*layers)

    def darknet53(num_classes):
        return Darknet53(DarkResidualBlock, num_classes)

In [2]:
model = Darknet53.darknet53(1000)

In [3]:
model = model.eval()

In [6]:
checkpoint = torch.load("model_best.pth.tar", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['state_dict'])